In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
from IPython.display import display

import matplotlib.pyplot as plt
import pywt
import scipy.stats

import datetime as dt
from collections import defaultdict, Counter

In [19]:
train_fold = '/content/gdrive/My Drive/Colab Notebooks/WUCI/UCI HAR Dataset/train/Inertial Signals/'
test_fold = '/content/gdrive/My Drive/Colab Notebooks/WUCI/UCI HAR Dataset/test/Inertial Signals/'

train = ['body_acc_x_train.txt', 'body_acc_y_train.txt', 'body_acc_z_train.txt', 
                     'body_gyro_x_train.txt', 'body_gyro_y_train.txt', 'body_gyro_z_train.txt',
                     'total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt']

test = ['body_acc_x_test.txt', 'body_acc_y_test.txt', 'body_acc_z_test.txt', 
                     'body_gyro_x_test.txt', 'body_gyro_y_test.txt', 'body_gyro_z_test.txt',
                     'total_acc_x_test.txt', 'total_acc_y_test.txt', 'total_acc_z_test.txt']

train_y_files = '/content/gdrive/My Drive/Colab Notebooks/WUCI/UCI HAR Dataset/train/y_train.txt'
test_y_files = '/content/gdrive/My Drive/Colab Notebooks/WUCI/UCI HAR Dataset/test/y_test.txt'



In [20]:
activities_description = {
    1: 'walking',
    2: 'walking upstairs',
    3: 'walking downstairs',
    4: 'sitting',
    5: 'standing',
    6: 'laying'
}

def signal_reader(filename):
    with open(filename, 'r') as fp:
        d = fp.read().splitlines()
        d = map(lambda x: x.rstrip().lstrip().split(), d)
        d = [list(map(float, line)) for line in d]
    return d

def label_reader(filename):        
    with open(filename, 'r') as fp:
        act = fp.read().splitlines()
        act = list(map(int, act))
    return act

def randomizer(dataset, labels):
    p = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[p, :, :]
    shuffled_labels = labels[p]
    return shuffled_dataset, shuffled_labels

In [21]:
arr_train_signals, arr_test_signals = [], []

for inp_file in train:
    signal = signal_reader(train_fold + inp_file)
    arr_train_signals.append(signal)
arr_train_signals = np.transpose(np.array(arr_train_signals), (1, 2, 0))

for inp_file in test:
    signal = signal_reader(test_fold + inp_file)
    arr_test_signals.append(signal)
arr_test_signals = np.transpose(np.array(arr_test_signals), (1, 2, 0))

labels_train = label_reader(train_y_files)
labels_test = label_reader(test_y_files)

[no_signals_train, no_steps_train, no_components_train] = np.shape(arr_train_signals)
[no_signals_test, no_steps_test, no_components_test] = np.shape(arr_train_signals)
no_labels = len(np.unique(labels_train[:]))

print("The train  has {} signals, each one of length {} and {} components ".format(no_signals_train, no_steps_train, no_components_train))
print("The test  has {} signals, each one of length {} and {} components ".format(no_signals_test, no_steps_test, no_components_test))
print("The train  has {} labels, with the following distribution:\n {}".format(np.shape(labels_train)[0], Counter(labels_train[:])))
print("The test  has {} labels, with the following distribution:\n {}".format(np.shape(labels_test)[0], Counter(labels_test[:])))

uci_har_signals_train, uci_har_labels_train = randomizer(arr_train_signals, np.array(labels_train))
uci_har_signals_test, uci_har_labels_test = randomizer(arr_test_signals, np.array(labels_test))

The train  has 7352 signals, each one of length 128 and 9 components 
The test  has 7352 signals, each one of length 128 and 9 components 
The train  has 7352 labels, with the following distribution:
 Counter({6: 1407, 5: 1374, 4: 1286, 1: 1226, 2: 1073, 3: 986})
The test  has 2947 labels, with the following distribution:
 Counter({6: 537, 5: 532, 1: 496, 4: 491, 2: 471, 3: 420})


In [22]:
print(uci_har_signals_train.shape)
print(uci_har_signals_test.shape)

(7352, 128, 9)
(2947, 128, 9)


In [23]:
dataset_size = len(uci_har_signals_train)
train_reshaped = uci_har_signals_train.reshape(dataset_size,-1)
train_reshaped.shape

(7352, 1152)

In [24]:
dataset_size = len(uci_har_signals_test)
test_reshaped = uci_har_signals_test.reshape(dataset_size,-1)
test_reshaped.shape

(2947, 1152)

#Training a kNN Classifier

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve, GridSearchCV
import time
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()
parameters = {'n_neighbors' : [6,7,8,9,10]}
model = GridSearchCV(knn,parameters,n_jobs=-1,verbose = 1)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.0min finished


182.69301342964172


In [ ]:
print('Best Validation Accuracy Score: '+ str(round(model.best_score_*100.0,2)) + ' %')

Best Validation Accuracy Score: 83.19 %


In [ ]:
best_neighbor = model.best_estimator_.get_params()['n_neighbors']
model = KNeighborsClassifier(n_neighbors = best_neighbor)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

0.8226757049560547


In [ ]:
ypred=model.predict(test_reshaped)
accuracy=accuracy_score(uci_har_labels_test,ypred)
c_report = classification_report(uci_har_labels_test,ypred)
print('Accuracy Score: '+ str(accuracy*100) + ' %')
print(c_report)

Accuracy Score: 64.50627757041059 %
              precision    recall  f1-score   support

           1       0.96      0.51      0.66       496
           2       0.96      0.53      0.68       471
           3       0.99      0.16      0.28       420
           4       0.59      0.79      0.67       491
           5       0.35      0.76      0.48       532
           6       1.00      1.00      1.00       537

    accuracy                           0.65      2947
   macro avg       0.81      0.63      0.63      2947
weighted avg       0.80      0.65      0.64      2947



# Training a Decision Tree Classifier

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier()
parameters = [{'criterion' : ['gini', 'entropy'],'min_samples_leaf' : [1, 5, 10]}]
model=GridSearchCV(classifier,parameters,n_jobs=-1,verbose=4)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.1min finished


147.01301288604736


In [ ]:
print('Best Validation Accuracy Score: '+ str(round(model.best_score_*100.0,2)) + ' %')

Best Validation Accuracy Score: 81.87 %


In [ ]:
best_criterion = model.best_estimator_.get_params()['criterion']
best_min_leaf = model.best_estimator_.get_params()['min_samples_leaf']
model = DecisionTreeClassifier(criterion = best_criterion, min_samples_leaf=best_min_leaf)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

21.307106256484985


In [ ]:
ypred=model.predict(test_reshaped)
accuracy=accuracy_score(uci_har_labels_test,ypred)
c_report = classification_report(uci_har_labels_test,ypred)
print('Accuracy Score: '+ str(accuracy*100) + ' %')
print(c_report)

Accuracy Score: 74.68612147947064 %
              precision    recall  f1-score   support

           1       0.61      0.66      0.63       496
           2       0.65      0.61      0.63       471
           3       0.65      0.63      0.64       420
           4       0.78      0.73      0.75       491
           5       0.76      0.80      0.78       532
           6       1.00      1.00      1.00       537

    accuracy                           0.75      2947
   macro avg       0.74      0.74      0.74      2947
weighted avg       0.75      0.75      0.75      2947



# Training a Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
parameters = {
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}
model=GridSearchCV(classifier,parameters,n_jobs=-1,cv=4,verbose=4)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 16.8min finished


1214.0718233585358


In [ ]:
print('Best Validation Accuracy Score: '+ str(round(model.best_score_*100.0,2)) + ' %')

Best Validation Accuracy Score: 96.31 %


In [ ]:
best_estimator = model.best_estimator_.get_params()['n_estimators']
best_max_features = model.best_estimator_.get_params()['max_features']
best_criterion = model.best_estimator_.get_params()['criterion']
model = RandomForestClassifier(n_estimators = best_estimator, max_features = best_max_features, criterion = best_criterion)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

207.01611495018005


In [ ]:
ypred=model.predict(test_reshaped)
accuracy=accuracy_score(uci_har_labels_test,ypred)
c_report = classification_report(uci_har_labels_test,ypred)
print('Accuracy Score: '+ str(accuracy*100) + ' %')
print(c_report)

Accuracy Score: 84.96776382762131 %
              precision    recall  f1-score   support

           1       0.81      0.90      0.85       496
           2       0.85      0.78      0.81       471
           3       0.85      0.87      0.86       420
           4       0.75      0.79      0.77       491
           5       0.84      0.75      0.79       532
           6       1.00      1.00      1.00       537

    accuracy                           0.85      2947
   macro avg       0.85      0.85      0.85      2947
weighted avg       0.85      0.85      0.85      2947



#Training a Gradient Boosting Classifier

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
parameters = {
    "learning_rate": [0.001, 0.01, 0.1],
    'n_estimators': [200, 500],
    "max_features":["log2","sqrt"]
    }

model=GridSearchCV(classifier,parameters,n_jobs=-1,cv=4,verbose=4)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 27.4min finished


1896.046844959259


In [ ]:
print('Best Validation Accuracy Score: '+ str(round(model.best_score_*100.0,2)) + ' %')

Best Validation Accuracy Score: 96.33 %


In [ ]:
best_learning_rate = model.best_estimator_.get_params()['learning_rate']
best_estimator = model.best_estimator_.get_params()['n_estimators']
best_max_features = model.best_estimator_.get_params()['max_features']
model = GradientBoostingClassifier(learning_rate=best_learning_rate,n_estimators=best_estimator,max_features=best_max_features)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

226.39382123947144


In [ ]:
ypred=model.predict(test_reshaped)
accuracy=accuracy_score(uci_har_labels_test,ypred)
c_report = classification_report(uci_har_labels_test,ypred)
print('Accuracy Score: '+ str(accuracy*100) + ' %')
print(c_report)

Accuracy Score: 88.49677638276214 %
              precision    recall  f1-score   support

           1       0.80      0.97      0.87       496
           2       0.90      0.78      0.83       471
           3       0.90      0.87      0.88       420
           4       0.87      0.80      0.83       491
           5       0.86      0.88      0.87       532
           6       1.00      1.00      1.00       537

    accuracy                           0.88      2947
   macro avg       0.89      0.88      0.88      2947
weighted avg       0.89      0.88      0.88      2947



#Training a SVM Classifier

In [30]:
import sklearn.svm
from sklearn.ensemble import GradientBoostingClassifier

In [32]:
classifier=sklearn.svm.SVC()
parameters=[{'kernel': ['rbf'], 'gamma': [0.01,0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
model=GridSearchCV(classifier,parameters,n_jobs=-1,cv=4,verbose=4)
import time
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed: 350.6min finished


21072.180725097656


In [33]:
print('Best Validation Accuracy Score: '+ str(round(model.best_score_*100.0,2)) + ' %')

Best Validation Accuracy Score: 93.61 %


In [34]:
best_gamma = model.best_estimator_.get_params()['gamma']
best_kernel = model.best_estimator_.get_params()['kernel']
best_C = model.best_estimator_.get_params()['C']

In [ ]:
model = SVC(kernel = best_kernel , C=best_C,gamma=best_gamma)
start = time.time()
model.fit(train_reshaped,uci_har_labels_train)
end = time.time()
print(end - start)

37.109313011169434


In [35]:
ypred=model.predict(test_reshaped)
accuracy=accuracy_score(uci_har_labels_test,ypred)
c_report = classification_report(uci_har_labels_test,ypred)
print('Accuracy Score: '+ str(accuracy*100) + ' %')
print(c_report)

Accuracy Score: 88.32711231761114 %
              precision    recall  f1-score   support

           1       0.92      0.94      0.93       496
           2       0.87      0.89      0.88       471
           3       0.85      0.92      0.88       420
           4       0.82      0.76      0.79       491
           5       0.82      0.83      0.83       532
           6       1.00      0.96      0.98       537

    accuracy                           0.88      2947
   macro avg       0.88      0.88      0.88      2947
weighted avg       0.88      0.88      0.88      2947

